In [1]:
from transformers import ViTImageProcessor, ViTForImageClassification
from PIL import Image
import requests
1
url = 'https://img.freepik.com/free-photo/grassy-field-with-leafless-trees-distance-cloudy-sky-background_181624-4535.jpg'
image = Image.open(requests.get(url, stream=True).raw)


labels = ['dew', 'fogsmog', 'frost', 'glaze', 'hail', 'lightning', 'rain', 'rainbow', 'rime', 'sandstorm', 'snow']
processor = ViTImageProcessor.from_pretrained('DunnBC22/vit-base-patch16-224-in21k-weather-images-classification',
                                              num_labels=len(labels),
    id2label={str(i): c for i, c in enumerate(labels)},
    label2id={c: str(i) for i, c in enumerate(labels)})
model = ViTForImageClassification.from_pretrained(
    'DunnBC22/vit-base-patch16-224-in21k-weather-images-classification',
    num_labels=len(labels),
    id2label={str(i): c for i, c in enumerate(labels)},
    label2id={c: str(i) for i, c in enumerate(labels)}
)
inputs = processor(images=image, return_tensors="pt")

outputs = model(**inputs)
# last_hidden_states = outputs.last_hidden_state
import torch

print(outputs)
idx = torch.argmax(outputs.logits)
print(f"idx is {idx}, value is {labels[idx]}")


/opt/anaconda3/envs/wtw/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/anaconda3/envs/wtw/lib/python3.8/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


ImageClassifierOutput(loss=None, logits=tensor([[-1.5074,  2.1681,  1.5576, -1.2671, -1.5802,  0.3297, -1.3380, -1.2168,
          1.7721, -0.0932,  0.1700]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
idx is 1, value is fogsmog


In [2]:
source1_data_path = "/Users/imdohun/PycharmProjects/wtw/AI/data/5-class_weather_status_image_classification"
source2_data_path = "/Users/imdohun/PycharmProjects/wtw/AI/data/weather_image_recognition"
merged_data_path = "/Users/imdohun/PycharmProjects/wtw/AI/data/merged_data"

In [3]:
from glob import glob
import random
random.seed(42)

In [4]:
source1_folders = glob(f"{source1_data_path}/*")
source2_folders = glob(f"{source2_data_path}/*")


In [5]:
source_folder2target_folder = {
    # source1 (11종) 
    "fogsmog": "foggy",
    "lightning": "rainy",
    "rain": "rainy",
    "sandstorm": "cloudy",
    "snow": "snowy",
    # source2 (5종)
    "cloudy": "cloudy",
    "foggy": "foggy",
    "rainy": "rainy",
    "snowy": "snowy",
    "sunny": "sunny",
}

In [6]:
source2folders = {
    "source1": source1_folders,
    "source2": source2_folders,
}

In [7]:
from shutil import copyfile


for source_name, folders in source2folders.items():
    for folder in folders:
        folder_name = folder.split("/")[-1]
        target_folder_name = source_folder2target_folder.get(folder_name, None)
        # 사용하지 않는 폴더의 경우 skip
        if target_folder_name is None:
            continue
        
        target_path = f"{merged_data_path}/{target_folder_name}"
        
        files = glob(f"{folder}/*")
        for file in files:
            file_name = file.split("/")[-1]
            file_name = f"{source_name}_{file_name}"
            copyfile(file, f"{target_path}/{file_name}")

In [14]:
folders = glob(f"{merged_data_path}/*")
for folder in folders:
    folder_name = folder.split("/")[-1]
    files = glob(f"{folder}/*")
    print(f"folder {folder_name}'s len is {len(files)}")

folder cloudy's len is 7394
folder sunny's len is 6274
folder rainy's len is 2830
folder snowy's len is 2496
folder foggy's len is 2112


In [16]:
import os
import random
import pandas as pd



classes = ["cloudy", "foggy", "rainy", "snowy", "sunny"]

meta_df = pd.DataFrame()
new_parent_dir = os.path.join(merged_data_path, "data")
for x in classes:
    list_of_file_names = os.path.join(merged_data_path, x)
    samples = os.listdir(list_of_file_names)
    samples = random.sample(samples, 1000)
    print(samples[0])
    for sample in samples:
        file_name = sample.split("/")[-1]
        copyfile(f"{merged_data_path}/{x}/{sample}", f"{merged_data_path}/data/{file_name}")
        
    temp_df = pd.DataFrame(samples, columns=['file_name'])
    temp_df['label'] = str(x)
    meta_df = pd.concat([meta_df, temp_df])

# Create new directory

os.makedirs(new_parent_dir, exist_ok=True)
where_to_save_meta_file = os.path.join(new_parent_dir, "metadata.csv")
meta_df.to_csv(where_to_save_meta_file, index=None)

source1_c0323.jpg
source1_4659.jpg
source1_rainy day-139.jpeg
source1_2236953011.jpg
source1_s2875.jpg


In [17]:
len(meta_df), len(glob(f"{'/Users/imdohun/PycharmProjects/wtw/AI/data/merged_data/data/*'}"))

(5000, 4951)

In [18]:
from PIL import ImageDraw, ImageFont, Image
import PIL.Image

from tqdm import tqdm

import numpy as np
import pandas as pd

import datasets
from datasets import load_dataset, Image, load_metric, DatasetDict, ClassLabel

import transformers
from transformers import Trainer, TrainingArguments
from transformers import ViTForImageClassification, ViTFeatureExtractor

import torch

import evaluate

In [19]:
dataset_path = f"{merged_data_path}/data"
dataset = load_dataset(dataset_path)

label_names = sorted(set(dataset["train"]["label"]))
print("lables: ", label_names)
dataset = dataset.cast_column("label", ClassLabel(names=label_names))

dataset

Generating train split: 4950 examples [00:00, 33235.32 examples/s]


lables:  ['cloudy', 'foggy', 'rainy', 'snowy', 'sunny']


Casting the dataset: 100%|██████████| 4950/4950 [00:00<00:00, 150712.15 examples/s]


DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 4950
    })
})

In [20]:
dataset = dataset.shuffle(seed=42)

train_split = dataset['train'].train_test_split(train_size=0.8)

ds = DatasetDict({
    'train' : train_split['train'],
    'test' : train_split['test']
})

In [21]:
print("Training Dataset")
print(ds['train'])
print(ds['train'][0])
print(ds['train'][-1])

print("Testing Dataset")
print(ds['test'])
print(ds['test'][0])
print(ds['test'][-1])

Training Dataset
Dataset({
    features: ['image', 'label'],
    num_rows: 3960
})
{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=500x733 at 0x172337B80>, 'label': 3}
{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=200x200 at 0x312EED2B0>, 'label': 4}
Testing Dataset
Dataset({
    features: ['image', 'label'],
    num_rows: 990
})
{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=500x328 at 0x312EED7C0>, 'label': 3}
{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=200x200 at 0x312EED5E0>, 'label': 0}


In [22]:
def show_grid_of_examples(ds, 
                          seed: int = 42, 
                          examples_per_class: int = 3, 
                          size=(350, 350)):
    '''
    This function displays a few pictures
    from each class in the dataset.
    '''
    w, h = size
    labels = ds['train'].features['label'].names
    grid = PIL.Image.new(mode='RGB', size=(examples_per_class * w, len(labels) * h))
    draw = ImageDraw.Draw(grid)
    font = ImageFont.truetype("Chalkduster.ttf", 24)
    
    for label_id, label in enumerate(labels):
        # filter the dataset by a single label, shuffle it, then grab a few samples
        ds_slice = ds['train'] \
                    .filter(lambda ex: ex['label'] == label_id) \
                    .shuffle(seed) \
                    .select(range(examples_per_class))
        
        # plot this label's examples in a row
        for i, example in enumerate(ds_slice):
            image = example['image']
            idx = examples_per_class * label_id + i
            box = (idx % examples_per_class * w, idx // examples_per_class * h)
            grid.paste(image.resize(size), box=box)
            draw.text(box, label, (255, 255, 255), font=font, dill=(0,0,255,1.0))
    
    return grid

In [23]:
MODEL_CKPT = 'google/vit-base-patch16-224-in21k'
MODEL_NAME=MODEL_CKPT + "-weather_images_classification"
NUM_OF_EPOCHS = 1

LEARNING_RATE = 2e-4
STEPS = 100

BATCH_SIZE = 16
DEVICE = torch.device("mps")

# REPORTS_TO = 'tensorboard'

In [24]:
feature_extractor = ViTFeatureExtractor.from_pretrained(MODEL_CKPT)

/opt/anaconda3/envs/wtw/lib/python3.8/site-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [25]:
def transform(sample_batch):
    # take a list of PIL images and turn them into pixel values
    inputs = feature_extractor([x.convert("RGB") for x in sample_batch['image']], return_tensors="pt")
    
    # prepare labels
    inputs['labels'] = sample_batch['label']
    return inputs

In [26]:
prepped_ds = ds.with_transform(transform)

In [27]:
def data_collator(batch):
    return {
        'pixel_values' : torch.stack([x['pixel_values'] for x in batch]),
        'labels' : torch.tensor([x['labels'] for x in batch])
    }

In [28]:
def compute_metrics(p):
    accuracy_metric = evaluate.load("accuracy")
    accuracy = accuracy_metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids)['accuracy']
    
    ### ------------------- F1 scores -------------------
    
    f1_score_metric = evaluate.load("f1")
    weighted_f1_score = f1_score_metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids, average='weighted')["f1"]
    micro_f1_score = f1_score_metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids, average='micro')['f1']
    macro_f1_score = f1_score_metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids, average='macro')["f1"]
    
    ### ------------------- recall -------------------
    
    recall_metric = evaluate.load("recall")
    weighted_recall = recall_metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids, average='weighted')["recall"]
    micro_recall = recall_metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids, average='micro')["recall"]
    macro_recall = recall_metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids, average='macro')["recall"]
    
    ### ------------------- precision -------------------
    
    precision_metric = evaluate.load("precision")
    weighted_precision = precision_metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids, average='weighted')["precision"]
    micro_precision = precision_metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids, average='micro')["precision"]
    macro_precision = precision_metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids, average='macro')["precision"]
    
    return {"accuracy" : accuracy, 
            "Weighted F1" : weighted_f1_score,
            "Micro F1" : micro_f1_score,
            "Macro F1" : macro_f1_score,
            "Weighted Recall" : weighted_recall,
            "Micro Recall" : micro_recall,
            "Macro Recall" : macro_recall,
            "Weighted Precision" : weighted_precision,
            "Micro Precision" : micro_precision,
            "Macro Precision" : macro_precision
            }

In [29]:
labels = dataset['train'].features['label'].names

model = ViTForImageClassification.from_pretrained(
    MODEL_CKPT,
    num_labels=len(labels),
    id2label={str(i): c for i, c in enumerate(labels)},
    label2id={c: str(i) for i, c in enumerate(labels)}
).to(DEVICE)

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [30]:
args = TrainingArguments(
    output_dir=MODEL_NAME,
    remove_unused_columns=False,
    num_train_epochs=NUM_OF_EPOCHS,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=BATCH_SIZE,
    learning_rate=LEARNING_RATE,
    disable_tqdm=False,
    load_best_model_at_end=True,
    metric_for_best_model="Weighted F1",
    logging_first_step=True,
    # hub_private_repo=True,
    # push_to_hub=True
)

/opt/anaconda3/envs/wtw/lib/python3.8/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [31]:
trainer = Trainer(
    model = model,
    args= args,
    data_collator = data_collator,
    compute_metrics = compute_metrics,
    train_dataset=prepped_ds['train'],
    eval_dataset=prepped_ds['test'],
    tokenizer=feature_extractor,
)

In [32]:
# !tmux new -s train
train_results = trainer.train()


  0%|          | 0/248 [00:00<?, ?it/s]

: 